# SFT on Reasoning Dataset for Qwen 2.5 0.5B

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
!pip install -q -U tensorboardX
!pip install -q wandb
!pip install -1 -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 7.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.9 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -1


In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from enum import Enum
from functools import partial
import pandas as pd
import json

from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, TaskType


In [ ]:
# Load dataset
dataset = load_dataset("ServiceNow-AI/R1-Distill-SFT",'v0')
# think_prompt = """You are a reasoning AI model solving complex problems by breaking into simple reasoning steps.before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>"""
def format_example(example):
    """Formats dataset example into a prompt-response format."""
    return {"text":f"Problem: {example['problem']}\nReasoning: {example['reannotated_assistant_content']}\nSolution: {example['solution']}"}

# dataset = dataset.map(lambda x: {"text": format_example(x)})
dataset = dataset.map(format_example,remove_columns = ['id', 'reannotated_assistant_content', 'problem', 'source', 'solution', 'verified', 'quality_metrics'])
dataset


README.md:   0%|          | 0.00/2.70k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/171647 [00:00<?, ? examples/s]

Map:   0%|          | 0/171647 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 171647
    })
})

In [ ]:
dataset = dataset['train'].train_test_split(0.0001)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 171629
    })
    test: Dataset({
        features: ['text'],
        num_rows: 18
    })
})

In [ ]:
class ChatmlSpecialTokens(str, Enum):
    think = "<think>"
    eothink = "</think>"
    pad_token = "<pad>"
    eos_token = "<eos>"
    @classmethod
    def list(cls):
        return [c.value for c in cls]



In [ ]:
# Load model and tokenizer
model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # Change to your base variantmodel
tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list()
    )
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             attn_implementation='eager',
                                             device_map="auto")
model.resize_token_embeddings(len(tokenizer))
model.to(torch.bfloat16)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151669, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2R

In [ ]:
from peft import LoraConfig

rank_dimension = 16
lora_alpha = 64
lora_dropout = 0.05

peft_config = LoraConfig(r=rank_dimension,
                         lora_alpha=lora_alpha,
                         lora_dropout=lora_dropout,
                         target_modules=["gate_proj","q_proj","lm_head","o_proj","k_proj","embed_tokens","down_proj","up_proj","v_proj"], # which layer in the transformers do we target ?
                         task_type=TaskType.CAUSAL_LM)

In [ ]:
# # Apply LoRA for efficient training
# lora_config = LoraConfig(
#     r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none",
#     task_type="CAUSAL_LM",
# )
# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()

trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
username="Navaneeth"    # REPLCAE with your Hugging Face username
output_dir = "Qwen2.5-1.5B-thinking-reasoning-model-V0" # The directory where the trained model checkpoints, logs, and other artifacts will be saved. It will also be the default name of the model when pushed to the hub if not redefined later.
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 4
logging_steps = 100
learning_rate = 1e-4 # The initial learning rate for the optimizer.

max_grad_norm = 1.0
num_train_epochs=1
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 1500
max_steps = 5000

training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=500,
    eval_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    bf16=True,
    hub_private_repo=False,
    push_to_hub=False,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    packing=True,
    max_seq_length=max_seq_length,
    max_steps = max_steps
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"].take(10000),
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/10000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/18 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/18 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/18 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/18 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

139

In [ ]:
model

In [ ]:
tokenizer.decode(model.generate(tokenizer("""Problem: Among the following given objects, those that can represent a set are ( )\nA: All very large numbers\nB: Numbers infinitely close to zero\nC: Smart people\nD: Real number roots of the equation $x^2 = 2$\nReasoning: """,return_tensors='pt').to('cuda').input_ids)[0])

'Problem: Among the following given objects, those that can represent a set are ( )\nA: All very large numbers\nB: Numbers infinitely close to zero\nC: Smart people\nD: Real number roots of the equation $x^2 = 2$\nReasoning: <think>\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n...\n'

In [ ]:
dataset['train'][0]['text']

'Problem: Among the following given objects, those that can represent a set are ( )\nA: All very large numbers\nB: Numbers infinitely close to zero\nC: Smart people\nD: Real number roots of the equation $x^2 = 2$\nReasoning: <think>\nOkay, so I have this multiple-choice question here about which of the given options can represent a set. The options are A, B, C, and D. Let me try to understand what each option is saying and whether they can be considered as sets.\n\nFirst, let\'s recall what a set is in mathematics. A set is a well-defined collection of distinct objects, which are called elements or members of the set. The key thing here is that the definition of a set must be clear and unambiguous so that we can determine whether a particular object is or isn\'t in the set.\n\nNow, let\'s look at each option one by one.\n\n**Option A: All very large numbers**\n\nHmm, "very large numbers." That sounds a bit vague. What exactly constitutes a "very large number"? Is it numbers greater tha

In [ ]:
# # Tokenization function
# def tokenize_function(example):
#     return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
# dataset = dataset.shuffle()
# dataset = dataset.train_test_split(0.1)
# subset  =dataset.take(10000)
# tokenized_datasets = subset.map(tokenize_function, batched=True)

In [ ]:
from trl import SFTConfig, SFTTrainer

In [ ]:
# Training arguments
training_args = SFTConfig(
    output_dir="./qwen-sft", per_device_train_batch_size=1, gradient_accumulation_steps=8,
    eval_strategy="log", save_strategy="epoch", logging_steps=10,max_steps = 1000,
    num_train_epochs=3, learning_rate=2e-5, weight_decay=0.01,
    fp16=True, save_total_limit=2, report_to="none"
)

# Trainer
trainer = SFTTrainer(
    model=model, args=training_args,
    train_dataset=tokenized_datasets,
    processing_class=tokenizer,
)


Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[0]['input_ids'])

'Problem: There were 27 boys and 35 girls on the playground at recess. There were _____ children on the playground at recess.\nReasoning: <think>\nFirst, I need to determine the total number of children on the playground by adding the number of boys and girls.\n\nThere are 27 boys and 35 girls.\n\nAdding these together: 27 boys + 35 girls = 62 children.\n\nTherefore, the total number of children on the playground is 62.\n</think>\n\nTo find the total number of children on the playground, we simply add the number of boys and girls together.\n\n\\[\n\\text{Total children} = \\text{Number of boys} + \\text{Number of girls}\n\\]\n\nPlugging in the given values:\n\n\\[\n\\text{Total children} = 27 \\text{ boys} + 35 \\text{ girls} = 62 \\text{ children}\n\\]\n\n**Final Answer:**\n\n\\[\n\\boxed{62}\n\\]\nSolution: \nThere were 62 children on the playground at recess. (27 boys + 35 girls = $\\boxed{62}$  children)'

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

152

In [ ]:
import os
os.environ["WANDB_DISABLED"]  ="true"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:

# Start training
trainer.train()


Step,Training Loss
10,3.359000
20,3.018400
30,3.277800
40,3.204200
50,3.253900
60,3.324900
70,3.337100
80,3.457900
90,3.216800
100,3.515600


KeyboardInterrupt: 

In [ ]:
#This cell only runs in our colab environment as I have added my huggingface token inside the environment key (HF_TOKEN)
trainer.push_to_hub()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.39M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sujithanumala/qwen-sft/commit/63133390c8df3f35a2f6907312144abeead52db5', commit_message='Qwen3B_ThinkingModel', commit_description='', oid='63133390c8df3f35a2f6907312144abeead52db5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sujithanumala/qwen-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='Sujithanumala/qwen-sft'), pr_revision=None, pr_num=None)

In [ ]:
text= dataset['train'][1000]['problem']

In [ ]:
output = tokenizer.decode(model.generate(tokenizer(text,return_tensors = 'pt').to('cuda')['input_ids'],max_new_tokens=500)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [ ]:
print(output)

Calculate: $3x^{2}y\cdot \left(-2xy\right)^{2}=\_\_\_\_\_\_$. To solve the expression \(3x^{2}y \cdot \left(-2xy\right)^{2}\), we will follow these steps:

1. Simplify the expression inside the parentheses.
2. Apply the exponent to the simplified expression.
3. Multiply the resulting expression by \(3x^{2}y\).

Let's start with the expression inside the parentheses: \(\left(-2xy\right)^{2}\). When we square a product, we square each factor individually. So, we have:
\[
\left(-2xy\right)^{2} = (-2)^2 \cdot x^2 \cdot y^2 = 4x^2y^2.
\]
Now, substitute this back into the original expression:
\[
3x^{2}y \cdot 4x^{2}y^{2}.
\]
Next, we multiply the coefficients and the variables separately. The coefficients are 3 and 4, so their product is:
\[
3 \cdot 4 = 12.
\]
For the variables, we use the property of exponents that states \(a^m \cdot a^n = a^{m+n}\). So, we have:
\[
x^2 \cdot x^2 = x^{2+2} = x^4,
\]
and
\[
y \cdot y^2 = y^{1+2} = y^3.
\]
Putting it all together, we get:
\[
12x^4y^3.
\]
The

In [ ]:
# actual_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [ ]:
base_output = tokenizer.decode(actual_model.generate(tokenizer(text,return_tensors = 'pt').to('cuda')['input_ids'],max_new_tokens=500)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [ ]:
print(base_output)

Calculate: $3x^{2}y\cdot \left(-2xy\right)^{2}=\_\_\_\_\_\_$. To solve the expression \(3x^2y \cdot (-2xy)^2\), we will follow these steps:

1. Simplify the expression inside the parentheses.
2. Apply the exponent to the simplified expression.
3. Multiply the resulting expression by \(3x^2y\).

Let's start with the expression inside the parentheses:

\[
(-2xy)^2
\]

When we square a product, we square each factor:

\[
(-2xy)^2 = (-2)^2 \cdot x^2 \cdot y^2 = 4x^2y^2
\]

Now we substitute this back into the original expression:

\[
3x^2y \cdot 4x^2y^2
\]

Next, we multiply the coefficients and the variables separately. The coefficients are 3 and 4, and the variables are \(x^2\) and \(y^2\):

\[
3 \cdot 4 = 12
\]
\[
x^2 \cdot x^2 = x^{2+2} = x^4
\]
\[
y \cdot y^2 = y^{1+2} = y^3
\]

Putting it all together, we get:

\[
12x^4y^3
\]

So the final answer is:

\[
\boxed{12x^4y^3}
\]<|endoftext|>


In [ ]:
output==base_output

False